![endpoints](../../images/endpoint.png)

In this lesson, we're going to take a look at how we can use a model that has been created in SageMaker. We will do this by first deploying our model. For us, this means **using SageMaker's functionality to create an endpoint that will be used as a way to send data to our model.**

For our purposes an endpoint is just a URL. Instead of returning a web page, like a typical url, this endpoint URL returns the results of performing inference. In addition, we are able to send data to this URL so that our model knows what to perform inference on. We won't go too far into the details of how this is all set up since SageMaker does most of the heavy lifting for us.

An important aspect that we will encounter is that SageMaker endpoints are secured. In this case, that means that only other AWS services with permission to access SageMaker endpoints can do so.

To start with, we won't need to worry about this too much since we will be working inside of a SageMaker notebook and so we will be able to access our deployed endpoints easily.

Later on we will talk about how to set things up so that a simple web app, which doesn't need to be given special permission, can access our SageMaker endpoint.

### Boston Housing - XGBoost (Deploy) - Low Level.ipynb

Using the low level approach to deploy our model requires us to create an endpoint, which will be used to send data to our model and to get inference results.

In order to create an endpoint in SageMaker, we first need to describe an endpoint configuration. This describes to SageMaker the various properties we want our endpoint to have. Once we've created the endpoint configuration we can ask SageMaker to create an endpoint with the properties we want.

The actual endpoint that is created by SageMaker is a combination of a compute instance (some remote server) running a docker container with the inference code on it and a URL that data can be sent to and returned from. This URL is used as an interface to the compute instance, which receives data, performs inference using our model and returns the result.

### IMDB Sentiment Analysis - XGBoost - Web App.ipynb

In this example we are going to look at how we can make use of a deployed model in a simple web app.

In order for our simple web app to interact with the deployed model we are going to have to solve a couple problems.

**The first obstacle:**
he endpoint that is created when we deploy a model using SageMaker is secured, meaning that only entities that are authenticated with AWS can send or receive data from the deployed model. This is a problem since authenticating for the purposes of a simple web app is a bit more work than we'd like.

**The second obstacle:**
The second obstacle is that our deployed model expects us to send it a review after it has been processed. That is, it assumes we have already tokenized the review and then created a bag of words encoding. However, we want our user to be able to type any review into our web app.

To solve these issues we are going to need to use some additional Amazon services. In particular, we are going to look at Amazon Lambda and API Gateway.

### Text Processing
[Bag-of-word Explanaion Video](https://www.youtube.com/watch?v=A7M1z8yLl0w&feature=emb_logo&ab_channel=Udacity)

I mentioned that one of our tasks will be to convert any user input text into data that our deployed model can see as input. You've seen a few examples of text pre-processing and the steps usually go something like this:

* Get rid of any special characters like punctuation
* Convert all text to lowercase and split into individual words
* Create a vocabulary that assigns each unique word a numerical value or converts words into a vector of numbers

This last step is often called word tokenization or vectorization.(One method bag of words).

### Endpoint
[Setting up endpoint for model](https://www.youtube.com/watch?v=WTwj-7XcTro&feature=emb_logo&ab_channel=Udacity)

[Setting up Lambda Function](https://www.youtube.com/watch?v=jOXETK4AerU&feature=emb_logo&ab_channel=Udacity)

[Setting up API Gateway](https://www.youtube.com/watch?v=AzBQ-aDQSG4&feature=emb_logo&ab_channel=Udacity)

[Setting up Web Application](https://www.youtube.com/watch?v=VgG41Q_a15I&feature=emb_logo&ab_channel=Udacity)

To deploy model and give acess to everyone, there are two obstacles we are going to need to overcome. The first is the security issue and the second is data processing. The way that we are going to approach solving these issues is by making use of Amazon Lambda and API Gateway.

### Setting up endpoint using Lambdas and Api Gateway
![api](../../images/lambda.png)

What this means is that when someone uses our web app, the following will occur:

1. To begin with, a user will type out a review and enter it into our web app.

2. Then, our web app will send that review to an endpoint that we created using API Gateway. This endpoint will be constructed so that anyone (including our web app) can use it.

3. API Gateway will forward the data on to the Lambda function

4. Once the Lambda function receives the user's review, it will process that review by tokenizing it and then creating a bag of words encoding of the result. After that, it will send the processed review off to our deployed model.

5. Once the deployed model performs inference on the processed review, the resulting sentiment will be returned back to the Lambda function.

6. Our Lambda function will then return the sentiment result back to our web app using the endpoint that was constructed using API Gateway.



**Review**
### Endpoint steps
1. You can start an endpoint by calling **.deploy()** on an estimator and passing in some information about the instance.

```xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')```

2. Then, you need to tell your endpoint, what type of data it expects to see as input (like .csv).

```from sagemaker.predictor import csv_serializer```

```xgb_predictor.content_type = 'text/csv'```

```xgb_predictor.serializer = csv_serializer```


3. Then, perform inference; you can pass some data as the "Body" of a message, to an endpoint and get a response back!

```response = runtime.invoke_endpoint(EndpointName = xgb_predictor.endpoint, ContentType = 'text/csv', Body = ','.join([str(val) for val in test_bow]).encode('utf-8'))```

4. The inference data is stored in the "Body" of the response, and can be retrieved:

```response = response['Body'].read().decode('utf-8')```

```print(response)```

5. Finally, do not forget to shut down your endpoint when you are done using it.

```xgb_predictor.delete_endpoint()```